<h1>Evaluation<h1>

<h5>분류의 성능 평가 지표</h5>

- 정확표(Accuracy)
- 오차행렬(Confusion Matrix)
- 정밀도(Precision)
- 재현율(Recall)
- F1 스코어
- ROC AUC

<br><br>

<h3>4.1 정확도(Accuracy)</h3>
<br><br>

$$
정확도(Accuracy) = \frac{예측 결과가 동일한 데이터 건수}{\text{전체 예측 데이터 건수}}
$$

<br><br>

- 직관적으로 모델 예측 성능 평가 지표<br>
    -> 실제 데이터에서 예측 데이터가 얼마나 같은지 판단하는 지표<br>
    
- 구성에 따라 ML 모델의 성능을 왜곡할 수 있으므로 정확도만 가지고 성능을 평가하진 않음


[예제]
- 사이킷런의 BaseEstimator 클래스를 상속 후 학습 진행 x
- Sex에 따라 생존자를 예측하는 단순한 Classifier 생성

** 사이킷런의 BaseEstimator 상속
-> Customized형태의 Estimator를 개발자가 생성할 수 있음

In [5]:
from sklearn.base import BaseEstimator
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

class MyDummyClassifier(BaseEstimator):
    # fit() 메서드는 아무것도 학습하지 않음
    def fit(self, x, y=None):
        pass
    
    # predict() 메서드는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측함
    def predict(self, x):
        pred = np.zeros( (x.shape[0], 1) )
        for i in range (x.shape[0]):
            if x['Sex'].iloc[i]==1:
                pred[i] = 0
            else:
                pred[i] = 1
                
        return pred.ravel()
    

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df = pd.read_csv('data_handling/titanic_train.csv')

titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].mean())
titanic_df['Cabin'] = titanic_df['Cabin'].fillna('N')
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('N')

titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
titanic_df.groupby(['Sex', 'Survived'])['Survived'].count()

# Null 처리 함수
def fillna(df):
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Cabin'] = df['Cabin'].fillna('N')
    df['Embarked'] = df['Embarked'].fillna('N')
    df['Fare'] = df['Fare'].fillna('N')
    # df['Age'].fillna(df['Age'].mean(), inplace=True)
    # df['Cabin'].fillna('N', inplace=True)
    # df['Embarked'].fillna('N', inplace=True)
    # df['Fare'].fillna('N', inplace=True)
    return df

# 머신러닝 알고즘 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df


y_titanic_df = titanic_df['Survived']
x_titanic_df = titanic_df.drop('Survived', axis=1)
x_titanic_df = transform_features(x_titanic_df)
x_train, x_test, y_train, y_test = train_test_split(x_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

myclf = MyDummyClassifier()
myclf.fit(x_train, y_train)

mypredictions = myclf.predict(x_test)
print('Dummy Clssifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test, mypredictions)))


Dummy Clssifier의 정확도는: 0.7877
